# Time zones 
🌐!

This is a demo of some convenience methods for manipulating timezones with the `Observer` class. The two methods are: 

* **`self.datetime_to_astropy_time(datetime)`** which converts a naive *or* timezone-aware datetime to an astropy.time.Time object. 
 * If the input datetime is naive, it assumes that the implied timezone is the one saved in the instance of `Observer` (in `self.timezone`).

* **`self.astropy_time_to_datetime(astropy_time)`** which converts an astropy.time.Time object into a localized datetime, in the timezone saved in the instance of `Observer` (in `self.timezone`).

In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import EarthLocation
import pytz
import datetime

from astroplan import Observer

# Set up an observer at ~Subaru
location = EarthLocation.from_geodetic(-155.4*u.deg, 19.8*u.deg)
obs = Observer(location=location, timezone=pytz.timezone('US/Hawaii'))

# Pick a local (Hawaii) time to observe: midnight
local_naive_datetime = datetime.datetime(2015, 7, 14, 0)

# What is the astropy.time.Time equivalent for this datetime?
astropy_time = obs.datetime_to_astropy_time(local_naive_datetime)
print('astropy.time.Time (UTC):', astropy_time)

astropy.time.Time (UTC): 2015-07-14 10:00:00


Convert that `astropy.time.Time` back to a localized datetime, arriving back at the original datetime (only this one is localized):

In [2]:
localized_datetime = obs.astropy_time_to_datetime(astropy_time)
print('datetime:', localized_datetime)
print('new datetime equivalent to original naive datetime?:', 
      local_naive_datetime == localized_datetime.replace(tzinfo=None))

datetime: 2015-07-14 00:00:00-10:00
new datetime equivalent to original naive datetime?: True


Let's say the Subaru observer is remotely observing from the East Coast. Let's convert their local time (Eastern) to an astropy time. Since this datetime is localized, `datetime_to_astropy_time` will use the datetime's timezone (rather than assuming `self.timezone`):

In [3]:
east_coast_datetime = pytz.timezone('US/Eastern').localize(datetime.datetime(2015, 7, 14, 6))
east_coast_astropy_time = obs.datetime_to_astropy_time(east_coast_datetime)
print('Convert local East Coast time to UTC:', east_coast_astropy_time)
print('Equivalent to original astropy time?:', east_coast_astropy_time == astropy_time)

Convert local East Coast time to UTC: 2015-07-14 10:00:00
Equivalent to original astropy time?: True


### Warning
How you construct your localized timezone is important! *Don't initialize datetimes with
the `tzinfo` kwarg.* Here's an example of when it doesn't work. These two times *should* be equal, but are not. See [pytz documentation](http://pytz.sourceforge.net/#localized-times-and-date-arithmetic) for discussion.

In [4]:
tzinfo_kwarg = datetime.datetime(2015, 7, 14, 6, tzinfo=pytz.timezone('US/Eastern'))
localized = pytz.timezone('US/Eastern').localize(datetime.datetime(2015, 7, 14, 6))
print('with tz assigned in kwarg:', tzinfo_kwarg)
print('with localization by tz.localize(dt):', localized)

with tz assigned in kwarg: 2015-07-14 06:00:00-04:56
with localization by tz.localize(dt): 2015-07-14 06:00:00-04:00
